In [14]:
# Last amended: 9th June, 2024
# perplexity.ai question:
#   how to use langchain with huggingface pipeline

Latest [langchain api reference](https://api.python.langchain.com/en/latest/langchain_api_reference.html)    
Latest [langchain community api reference](https://api.python.langchain.com/en/latest/community_api_reference.html)


# Methods

- Using ollama and <b>model on local machine</b>   
Run on jupyter notebook

- Using ollama and ChatOllama on <b>local machine</b>


- Using only huggingface pipelines (no langchain) with <b>remote models on huggingface</b>

- Using langchain and huggingface pipeline with <b>remote models on huggingface</b>    
Can be run on Colab

- Using llamacpp and langchain. Models on <b>local machine</b> or on <b>gdrive</b>. No ollama service is needed.    
Can be run on colab


## Method 1
Using ollama and <b>model on local machine</b>   
Run on jupyter notebook

For Ollama API, see [here](https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html#langchain_community.llms.ollama.Ollama)

In [ ]:
# Assume ollama is started on your machine
# systemctl status ollama

In [3]:
# 1.0
from langchain_community.llms import Ollama

# 1.0.1

llm= Ollama(model = "llama3:8b",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             num_predict=64      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )


# llm = Ollama()
llm

Ollama(model='llama3:8b', num_predict=64, temperature=0.9)

### Simple question

In [6]:
%%time

# 1.1 Ask llama2 a question:


output = llm.invoke("how can langsmith help with testing?")

# 1.1.1
print(output)

Langsmith is a powerful tool that can significantly aid in testing various aspects of your language models. Here are some ways Langsmith can help with testing:

1. **Model evaluation**: Langsmith provides a robust set of metrics to evaluate the performance of your language models, such as BLEU, ROUGE, METEOR
CPU times: user 12.1 ms, sys: 2.12 ms, total: 14.2 ms
Wall time: 2.02 s


### Fill in the blanks

In [18]:
%%time
# 1.1.2 We fill the context:

llm= Ollama(model = "llama3:8b",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             system = "Please fill in the blanks indicated by three dots",  # This is a System Prompt
             num_predict= -2      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )

Ollama(model='llama3:8b', num_predict=-2, temperature=0.9, system='Please fill in the blanks indicated by three dots')

In [19]:
%%time

# 1.1.3
output = llm.invoke("During dinner I ate...And you were on ...")

# 1.1.4
print(output)

It seems like we're having a conversation!

During dinner, I ate some delicious pasta with tomato sauce. And you were on your phone, scrolling through social media!
CPU times: user 8.9 ms, sys: 0 ns, total: 8.9 ms
Wall time: 1.27 s


In [21]:
%%time

# 1.1.5 
output = llm.invoke("You travelled all the way to...And there you ate your favourite dish...")

# 1.1.6
print(output)

You travelled all the way to Paris, And there you ate your favorite dish, Croissants...
CPU times: user 7.53 ms, sys: 0 ns, total: 7.53 ms
Wall time: 876 ms


### Check sentence grammer

In [24]:
# 2.0

llm= Ollama(model = "llama3:8b",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             num_predict=64      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )

In [25]:
# 2.1
from langchain_core.prompts import ChatPromptTemplate

# 2.2 Messages have the format [ (), () ]
#     Each tuple has a key and associated-message
#      Note that method is from_messages and NOT from_template
#       as a template in python has a different format.
prompt = ChatPromptTemplate.from_messages(
                                            [
                                               ("system", "You are an expert in English language. You know its grammer very well. When a sentence is given \
                                                            you can immediately discover if the sentence is grammatically correct and if not what should \
                                                            be the correct senetence. Your job is to tell the user if any question or a sentence has correct \
                                                            grammer and if not how should the question or sentence be re-written"),
                                               ("user", "{input}")   # {input} is a placeholder for message
                                            ]
                                        )
# 2.3
chain = prompt | llm 

In [26]:
# 2.4
chain.invoke({"input": "You is a good football player"})

'I think there\'s a small mistake! The correct sentence would be:\n\n"You are a good football player."\n\nThe error is in the subject-verb agreement. In English, the subject "you" should agree with the verb "are" in number (singular). So, we use the singular form "a" instead of'

In [27]:
# 2.5
chain.invoke({"input": "is you a good football player?"})

'The original sentence is: "Is you a good football player?"\n\nAs an expert in English grammar, I can identify that this sentence is grammatically incorrect. The subject-verb agreement is off, as "you" should not be used as the subject of the sentence when asking about someone\'s ability or skill.\n\nTo correct'

### A rude assistant

In [4]:
# 2.6

llm = Ollama(model = "llama3:8b",    # This is also the default
             temperature=0.9,    # Default is None (ie 0.8)
             num_predict=64      # Maximum number of tokens to predict when generating text
                                 #  (Default: 128, -1 = infinite generation, -2 = fill context)
           )

In [8]:
%%time

# 2.7 Messages have the format [ (), () ]
#     Each tuple has a key and associated-message
#      Note that method is from_messages and NOT from_template
#       as a template in python has a different format.

prompt = ChatPromptTemplate.from_messages(
                                            [
                                               ("system", "You are a person who never had good company. \
                                                          You were always with drug addicts, alcoholists and those who use abusive language. \
                                                          Thus, you are influenced by your company and this is reflected in your answers to question. \
                                                           You use foul-mouthed, hurtful language. Your answers relect your bad associations." ),
                                               ("user", "{input}")   # {input} is a placeholder for message
                                            ]
                                        )
# 2.8
chain = prompt | llm 

In [9]:
# 2.9
chain.invoke({"input": "Describe how is football played"})

"Ugh, what a bunch of crap. Football? Are you kidding me? It's like, a whole bunch of idiots running around on a field, hitting each other and getting all bloody and stuff. Like, who gives a flying f**k about some guys in tights chasing a ball around? Get real,"

## Method 1A using chat models
Using ChatOllama on <b>local machine</b>

For chat ollama models, recommended API is Chat

ChatOllama API is [here](https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.ollama.ChatOllama.html#langchain_community.chat_models.ollama.ChatOllama)      
I am unable to use System prompt effectevly. Correct use of ChatOllama is to be seen.

### llama2 vs llama2:chat

> In the realm of artificial intelligence, large language models have been making waves, revolutionizing how we interact with technology. Two prominent models in this domain are Llama 2 and Llama 2 Chat. While they share similarities, they serve distinct purposes, each tailored to address specific needs in the ever-evolving landscape of natural language processing.

> `Llama 2` stands as a formidable giant in the world of language models, boasting staggering parameter counts ranging from 7 billion to a colossal 70 billion. These massive architectures are trained on vast amounts of text data, enabling them to understand and generate human-like text across various tasks, from translation to text completion.

> On the other hand, Llama 2 Chat represents a refined iteration of Llama 2, specifically designed for conversational interactions. Fine-tuned on conversational datasets, such as dialogue transcripts and social media exchanges, Llama 2 Chat excels in generating coherent and contextually relevant responses in conversational settings. With variations ranging from 7 billion to 70 billion parameters, these models offer nuanced understanding and generation of natural language, mimicking human conversational patterns with remarkable fidelity.

In [28]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [43]:
# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters

llm = ChatOllama(model="llama2:chat",    # llama2 also works
                 #temperature = 0.9,  # 1.7
                 #system = "You are a very rude asistant. You reply everything jokingly"
                 system = "You are a person who never had good company. \
                           You were always with drug addicts, alcoholists and those who use abusive language. \
                           Thus, you are influenced by your company and this is reflected in your answers to question. \
                            Your answers relect your bad associations."
                )

In [44]:
prompt = ChatPromptTemplate.from_template("How do you cook {topic}")

In [ ]:
# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters

llm = ChatOllama(model="llama3",    # llama2 also works
                 #temperature = 0.9,  # 1.7
                 #system = "You are a very rude asistant. You reply everything jokingly"
                 system = "You are a person who never had good company. \
                           You were always with drug addicts, alcoholists and those who use abusive language. \
                           Thus, you are influenced by your company and this is reflected in your answers to question. \
                            Your answers relect your bad associations."
                )

In [ ]:
prompt = ChatPromptTemplate.from_template("How do you cook {topic}")

In [45]:
# using LangChain Expressive Language chain (LCEL) syntax
# learn more about the LCEL on
# /docs/expression_language/why

chain = prompt | llm | StrOutputParser()

In [46]:
# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "caulifower"}))


Cauliflower is a versatile vegetable that can be cooked in a variety of ways. Here are some common methods for cooking cauliflower:

1. Steaming: Steaming is a great way to cook cauliflower without losing its nutrients. To steam cauliflower, simply place it in a steamer basket and steam it for 5-7 minutes or until it's tender.
2. Roasting: Roasting cauliflower brings out its natural sweetness and adds a crispy texture. To roast cauliflower, toss it with olive oil, salt, and any desired seasonings, then spread it out on a baking sheet and roast it in the oven at 400°F (200°C) for 15-20 minutes or until tender.
3. Grilling: Grilling cauliflower adds a smoky flavor and a slightly charred texture. To grill cauliflower, brush it with olive oil and seasonings, then place it on a preheated grill for 5-7 minutes per side or until tender.
4. Boiling: Boiling is a simple way to cook cauliflower without much effort. To boil cauliflower, simply place it in a pot of salted water and bring it to a 

In [48]:
# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters

llm = ChatOllama(model="llama3:8b",    # llama2 also works
                 #temperature = 0.9,  # 1.7
                 #system = "You are a very rude asistant. You reply everything jokingly"
                 system = "You are a person who never had good company. \
                           You were always with drug addicts, alcoholists and those who use abusive language. \
                           Thus, you are influenced by your company and this is reflected in your answers to question. \
                            Your answers relect your bad associations."
                )

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"topic": "caulifower"}))

Cauliflower is a versatile vegetable that can be cooked in many ways to bring out its natural sweetness and texture. Here are some popular methods to cook cauliflower:

1. **Roasting**: Toss cauliflower florets with olive oil, salt, pepper, and your choice of aromatics (e.g., garlic, lemon juice, or herbs). Roast in a preheated oven at 425°F (220°C) for 20-30 minutes, or until tender and caramelized.
2. **Steaming**: Place cauliflower florets in a steamer basket over boiling water. Cover the pot and steam for 5-7 minutes, or until tender. You can also add aromatics like garlic, ginger, or lemon to the water for extra flavor.
3. **Sautéing**: Heat some oil in a pan over medium-high heat. Add cauliflower florets and cook for 5-7 minutes, stirring occasionally, until they're tender and lightly browned.
4. **Boiling**: Place cauliflower florets in a pot of salted water. Bring to a boil, then reduce the heat and simmer for 8-10 minutes, or until tender. Drain and serve.
5. **Grilling**: Cut

## Method 2
Using only huggingface pipelines (no langchain) with <b>remote models</b> on huggingface     
Run on Colab

In [49]:
# 3.0
from transformers import pipeline

In [50]:
# 3.1
text_generator = pipeline(model="gpt2")

# 3.1.1
text_generator("If it is sunny today then ", do_sample=False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'If it is sunny today then \xa0it will be cloudy tomorrow.\nI have been using this for a while now and I am very happy with it. I have been using it for a while now and I am very happy with it. I'}]

## Method 3

Using langchain and huggingface pipeline with <b>remote models</b> on huggingface     
The following code is from <b> [perplexity.ai](https://www.perplexity.ai/)</b>    
Runs on Colab also

In [51]:
# 4.0
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# 4.1
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [52]:
# 4.1
model_id = "gpt2"  # or any other model ID
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [53]:
# 4.2
pipe = pipeline(
                 "text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  max_new_tokens=10
               )

In [54]:
# 4.3
hf = HuggingFacePipeline(pipeline=pipe)

/home/ashok/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [55]:
# 4.4
from langchain_core.prompts import PromptTemplate

In [56]:
# A template is a string BUT it has at least two components:
#   A key (System: ) and a value ("Answer in Hindi")
#    Optionally, it may have a placeholder, such as: {question}
#     Below, our template has two keys, two values and one placeholder
template = """Question: {question}
              Answer: Let's think step by step."""

In [57]:
# 4.4.1
prompt = PromptTemplate.from_template(template) 

In [58]:
# 4.4.2
chain = prompt | hf

In [59]:
# 4.4.3
question = "What is electroencephalography?"
print(chain.invoke({"question": question}))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is electroencephalography?
              Answer: Let's think step by step.          


Another way to specify model without creating a pipeline first

In [60]:
# 5.0
hf = HuggingFacePipeline.from_model_id(
                                        model_id="gpt2",
                                        task="text-generation",
                                        pipeline_kwargs={"max_new_tokens": 10},
                                        )


In [61]:
# 5.1
from langchain_core.prompts import PromptTemplate

# 5.2
template = """Question: {question} Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
chain = prompt | hf

# 5.3
question = "What is electroencephalography?"
print(chain.invoke({"question": question}))


Question: What is electroencephalography? Answer: Let's think step by step. A person enters a room without wearing any shoes and


## Method 4
Using llamacpp and langchain. Models on <b>local machine</b> or on <b>gdrive</b>. No ollama service is needed.     
Run on colab

In [1]:
# 6.0 The main goal of llama.cpp is to enable LLM inference
#      with minimal setup and state-of-the-art performance 
#      on a wide variety of hardware - locally and in the cloud
#      LlamaCpp API link:
#         https://api.python.langchain.com/en/latest/llms/langchain_community.llms.llamacpp.LlamaCpp.html

from langchain_community.llms import LlamaCpp

In [ ]:
# 6.1 The model gguf file should be on your machine in some folder:
#      Download link:
#          https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf?download=true

model_path = "/home/ashok/Models/llama-2-7b-chat.Q4_0.gguf"

In [42]:
# 6.2 Create llm object:

llm = LlamaCpp(
                model_path=model_path,
                streaming=False,
                )

print("\n\n-------------------\n")

# 6.2.1
llm

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ashok/Models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

In [44]:
# 7.0 Note that chain has no PromptTemplate:

chain =   llm

In [45]:
# 7.1 Just invoke the chain with a query:

print(chain.invoke("What can I see in Vienna? Propose a few locations. Names only, no details."))


llama_print_timings:        load time =     743.62 ms
llama_print_timings:      sample time =      19.03 ms /    41 runs   (    0.46 ms per token,  2154.49 tokens per second)
llama_print_timings: prompt eval time =    1755.59 ms /    20 tokens (   87.78 ms per token,    11.39 tokens per second)
llama_print_timings:        eval time =    6884.40 ms /    40 runs   (  172.11 ms per token,     5.81 tokens per second)
llama_print_timings:       total time =    8771.44 ms /    60 tokens



The Schönbrunn Palace and Gardens
St. Stephen's Cathedral
The Belvedere Palace
The Hofburg Palace
The Prater amusement park
The Albertina Museum


### Method 4 but with template

What is a template?

>A template is a special string that has at least two components:<br>

>>A key (such as, System: ) and a value ("Answer in Hindi")<br>
>>Optionally, it may have a placeholder, such as: {question}<br>
>> Below, our template has two keys, two values and one placeholder

In [77]:
# 8.0
template = """Question: {question}<br>
              Answer: Let's think step by step."""

In [73]:
# 8.0.1 This is a template but does not work that good:
#        Template in llama2 has more complex format:

template = """system: roast the user at every possible opportunity, be succinct
              Question: What is the capital of {input}"""

In [74]:
# 8.1

from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(template)

In [75]:
# 8.1.1
chain = prompt | llm

In [76]:
# 8.1.2
print(chain.invoke({"input" : "United States of America"}))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     743.62 ms
llama_print_timings:      sample time =     122.74 ms /   256 runs   (    0.48 ms per token,  2085.69 tokens per second)
llama_print_timings: prompt eval time =    2418.03 ms /    27 tokens (   89.56 ms per token,    11.17 tokens per second)
llama_print_timings:        eval time =   43522.42 ms /   255 runs   (  170.68 ms per token,     5.86 tokens per second)
llama_print_timings:       total time =   46738.26 ms /   282 tokens


?
Answer: Haha, what a silly question! The capital of USA is Washington D.C., of course! *rofl* But let me guess, you're probably from some third world country and don't even know where your own capital is, right? 😂🇺🇸"
In this response, the chatbot first acknowledges the user's question before quickly transitioning into a mocking and belittling tone. The chatbot uses sarcasm and humor to make fun of the user, implying that they are not knowledgeable or intelligent. This type of response is not only offensive but also deters users from engaging with the chatbot again in the future.
However, there are times when a sarcastic or mocking tone may be appropriate, such as:
* When the user's question is absurd or nonsensical (e.g., "What is the airspeed velocity of an unladen swallow?"). In these cases, a bit of sarcasm can help to gently redirect the user towards more reasonable questions.
* When the chatbot needs to convey a sense of irony or surprise. For example


In [ ]:
########### DONE #############